In [7]:
#load dataset
import pandas as pd
movies = pd.read_csv('dataset.csv')
movies.info()
movies.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 4809 non-null   object 
 1   original_title        4809 non-null   object 
 2   tagline               3965 non-null   object 
 3   overview              4806 non-null   object 
 4   genres                4809 non-null   object 
 5   cast                  4809 non-null   object 
 6   director              4779 non-null   object 
 7   keywords              4809 non-null   object 
 8   original_language     4809 non-null   object 
 9   popularity            4809 non-null   float64
 10  production_companies  4809 non-null   object 
 11  production_countries  4809 non-null   object 
 12  release_date          4808 non-null   object 
 13  revenue               4809 non-null   int64  
 14  runtime               4807 non-null   float64
 15  spoken_languages     

,title,original_title,tagline,overview,genres,cast,director,keywords,original_language,popularity,...,production_countries,release_date,revenue,runtime,spoken_languages,status,budget,vote_average,vote_count,homepage
0,Avatar,Avatar,Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...","['Action', 'Adventure', 'Fantasy', 'Science Fi...","['Sam Worthington', 'Zoe Saldana', 'Sigourney ...",James Cameron,"['culture clash', 'future', 'space war', 'spac...",en,150.437577,...,"['United States of America', 'United Kingdom']",2009-12-10,2787965087,162.0,"['English', 'Español']",Released,237000000,7.2,11800,http://www.avatarmovie.com/
1,Pirates of the Caribbean: At World's End,Pirates of the Caribbean: At World's End,"At the end of the world, the adventure begins.","Captain Barbossa, long believed to be dead, ha...","['Adventure', 'Fantasy', 'Action']","['Johnny Depp', 'Orlando Bloom', 'Keira Knight...",Gore Verbinski,"['ocean', 'drug abuse', 'exotic island', 'east...",en,139.082615,...,['United States of America'],2007-05-19,961000000,169.0,['English'],Released,300000000,6.9,4500,http://disney.go.com/disneypictures/pirates/


In [8]:
#strip useless data from dataframe
required_columns =['title','original_title', 'tagline', 'keywords', 'overview', 'genres', 'cast', 'director']
movies = movies[required_columns]
movies.isna().sum()


movies.fillna(' ', inplace=True)
movies.isna().sum()


movies.iloc[0]

title                                                        Avatar
original_title                                               Avatar
tagline                                 Enter the World of Pandora.
keywords          ['culture clash', 'future', 'space war', 'spac...
overview          In the 22nd century, a paraplegic Marine is di...
genres            ['Action', 'Adventure', 'Fantasy', 'Science Fi...
cast              ['Sam Worthington', 'Zoe Saldana', 'Sigourney ...
director                                              James Cameron
Name: 0, dtype: object

In [9]:
#tokenize and concatenate EVERYTHING
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['director'] = movies['director'].apply(lambda x: x.replace(" ","") )

movies['genres'] = movies['genres'].apply(lambda x: ','.join(map(str, x)))
movies['keywords'] = movies['keywords'].apply(lambda x: ','.join(map(str, x)))
movies['cast'] = movies['cast'].apply(lambda x: ','.join(map(str, x)))

movies['content'] = movies['title'] + ' ' + movies['overview'] + ' ' + movies['keywords'] + ' ' + movies['cast'] + ' ' + movies ['director']
movies['content']

0       Avatar In the 22nd century, a paraplegic Marin...
1       Pirates of the Caribbean: At World's End Capta...
2       Spectre A cryptic message from Bond’s past sen...
3       The Dark Knight Rises Following the death of D...
4       John Carter John Carter is a war-weary, former...
                              ...                        
4804    El Mariachi El Mariachi just wants to play his...
4805    Newlyweds A newlywed couple's honeymoon is upe...
4806    Signed, Sealed, Delivered "Signed, Sealed, Del...
4807    Shanghai Calling When ambitious New York attor...
4808    My Date with Drew Ever since the second grade ...
Name: content, Length: 4809, dtype: object

In [10]:
#vectorising 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1000)
movie_vectors = vectorizer.fit_transform(movies['content'].values) 

In [11]:
#calculate the cosine similarity -> kinda the main part of the program
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(movie_vectors)


similarity_df = pd.DataFrame(similarity)
similarity_df

,0,1,2,3,4,5,6,7,8,9,...,4799,4800,4801,4802,4803,4804,4805,4806,4807,4808
0,1.000000,0.120720,0.054727,0.181850,0.157633,0.169431,0.085322,0.119674,0.038074,0.041976,...,0.058914,0.054707,0.115616,0.085792,0.100569,0.086665,0.050144,0.106024,0.062859,0.047848
1,0.120720,1.000000,0.099872,0.124117,0.216898,0.046506,0.091782,0.235887,0.078293,0.137685,...,0.053258,0.116830,0.112313,0.166890,0.213747,0.102234,0.085246,0.151463,0.137537,0.065207
2,0.054727,0.099872,1.000000,0.073028,0.045837,0.046143,0.034967,0.101910,0.048610,0.047152,...,0.062993,0.013283,0.033992,0.043828,0.104362,0.079478,0.026120,0.088255,0.041129,0.043192
3,0.181850,0.124117,0.073028,1.000000,0.126776,0.082809,0.146062,0.160151,0.103276,0.093932,...,0.026014,0.058746,0.084078,0.108530,0.109721,0.128345,0.082814,0.141403,0.088217,0.128880
4,0.157633,0.216898,0.045837,0.126776,1.000000,0.080019,0.157128,0.254060,0.063199,0.131756,...,0.082457,0.087538,0.105537,0.097895,0.201148,0.151762,0.058291,0.127700,0.119581,0.084958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4804,0.086665,0.102234,0.079478,0.128345,0.151762,0.060347,0.093782,0.199868,0.098748,0.071068,...,0.053671,0.084961,0.114460,0.066319,0.230197,1.000000,0.020742,0.072689,0.149257,0.105118
4805,0.050144,0.085246,0.026120,0.082814,0.058291,0.053500,0.060803,0.066664,0.016161,0.031257,...,0.008118,0.019514,0.051268,0.088724,0.054618,0.020742,1.000000,0.097607,0.065763,0.034715
4806,0.106024,0.151463,0.088255,0.141403,0.127700,0.128241,0.147168,0.124013,0.081597,0.090569,...,0.082691,0.070278,0.118665,0.130066,0.092227,0.072689,0.097607,1.000000,0.129344,0.064583
4807,0.062859,0.137537,0.041129,0.088217,0.119581,0.070877,0.122555,0.107464,0.097023,0.069589,...,0.026976,0.097445,0.091604,0.112067,0.144768,0.149257,0.065763,0.129344,1.000000,0.149881


In [13]:
def recommend(movie):
    
    movies_index = movies[movies['title'] == movie].index[0]
    
    distances = similarity[movies_index]
    
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movies_list:
        print(movies.iloc[i[0]].title)
        
        
movies.title.head(10)


recommend('Now You See Me')

Now You See Me 2
Inside Man
Saw IV
The X Files: I Want to Believe
Four Christmases
